In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
btc_data = pd.read_csv('Gemini_BTCUSD_d.csv', skiprows=1)
btc_data = btc_data.iloc[:, 1:]
btc_data['date'] = pd.to_datetime(btc_data['date']).dt.date
btc_data = btc_data.sort_values('date', ascending=True)

rf_data = pd.read_csv('Risk_Free.csv')
rf_data['date'] = pd.to_datetime(rf_data['date']).dt.date

data = pd.merge(btc_data, rf_data, on='date', how='inner')
data = data.sort_values(by='date').reset_index(drop=True)
data


,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf
0,2015-10-08,BTC/USD,0.00,245.00,0.00,243.60,34.754703,8.466246e+03,0.000007
1,2015-10-09,BTC/USD,243.60,249.97,243.60,245.51,61.587068,1.512024e+04,0.000007
2,2015-10-10,BTC/USD,245.51,246.30,244.60,246.30,30.870549,7.603416e+03,0.000007
3,2015-10-11,BTC/USD,246.30,249.50,245.96,248.98,22.747091,5.663571e+03,0.000007
4,2015-10-12,BTC/USD,248.98,248.98,245.75,245.75,71.047743,1.745998e+04,0.000007
...,...,...,...,...,...,...,...,...,...
3670,2024-11-06,BTC/USD,69381.93,76500.00,69273.86,75639.97,3046.413972,2.304307e+08,0.000137
3671,2024-11-07,BTC/USD,75639.97,76954.22,74484.13,75921.27,1086.378106,8.247921e+07,0.000130
3672,2024-11-08,BTC/USD,75921.27,77314.00,75655.78,76570.01,1173.097288,8.982407e+07,0.000130
3673,2024-11-09,BTC/USD,76570.01,76944.00,75763.62,76719.63,355.137546,2.724602e+07,0.000130


In [6]:
# create the signal for the trading range breakout
lookback_period = 50 # tracking back for 50 days

data['SUP'] = data['close'].shift(1).rolling(window=lookback_period).min()
data['RES'] = data['close'].shift(1).rolling(window=lookback_period).max()
data

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SUP,RES
0,2015-10-08,BTC/USD,0.00,245.00,0.00,243.60,34.754703,8.466246e+03,0.000007,NaN,NaN
1,2015-10-09,BTC/USD,243.60,249.97,243.60,245.51,61.587068,1.512024e+04,0.000007,NaN,NaN
2,2015-10-10,BTC/USD,245.51,246.30,244.60,246.30,30.870549,7.603416e+03,0.000007,NaN,NaN
3,2015-10-11,BTC/USD,246.30,249.50,245.96,248.98,22.747091,5.663571e+03,0.000007,NaN,NaN
4,2015-10-12,BTC/USD,248.98,248.98,245.75,245.75,71.047743,1.745998e+04,0.000007,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3670,2024-11-06,BTC/USD,69381.93,76500.00,69273.86,75639.97,3046.413972,2.304307e+08,0.000137,60280.67,72700.31
3671,2024-11-07,BTC/USD,75639.97,76954.22,74484.13,75921.27,1086.378106,8.247921e+07,0.000130,60280.67,75639.97
3672,2024-11-08,BTC/USD,75921.27,77314.00,75655.78,76570.01,1173.097288,8.982407e+07,0.000130,60280.67,75921.27
3673,2024-11-09,BTC/USD,76570.01,76944.00,75763.62,76719.63,355.137546,2.724602e+07,0.000130,60280.67,76570.01


In [8]:
data = data.dropna()

data['signal'] = np.where(data['close'] > data['RES'], 1, 0)  # Buy signal
data['signal'] = np.where(data['close'] < data['SUP'], -1, data['signal'])  # Sell signal

data

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SUP,RES,signal
50,2015-11-27,BTC/USD,350.82,369.04,348.79,353.78,1084.180175,3.835613e+05,0.000007,243.60,441.95,0
51,2015-11-28,BTC/USD,353.78,359.99,352.53,357.01,101.355031,3.618476e+04,0.000007,245.51,441.95,0
52,2015-11-29,BTC/USD,357.01,380.00,354.01,373.87,683.993384,2.557246e+05,0.000007,245.75,441.95,0
53,2015-11-30,BTC/USD,373.87,380.99,367.53,377.32,1346.303446,5.079872e+05,0.000007,245.75,441.95,0
54,2015-12-01,BTC/USD,377.32,379.99,349.00,352.52,545.953655,1.924596e+05,0.000007,245.75,441.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3670,2024-11-06,BTC/USD,69381.93,76500.00,69273.86,75639.97,3046.413972,2.304307e+08,0.000137,60280.67,72700.31,1
3671,2024-11-07,BTC/USD,75639.97,76954.22,74484.13,75921.27,1086.378106,8.247921e+07,0.000130,60280.67,75639.97,1
3672,2024-11-08,BTC/USD,75921.27,77314.00,75655.78,76570.01,1173.097288,8.982407e+07,0.000130,60280.67,75921.27,1
3673,2024-11-09,BTC/USD,76570.01,76944.00,75763.62,76719.63,355.137546,2.724602e+07,0.000130,60280.67,76570.01,1
